In [1]:
import pandas as pd
from IPython.display import Image
import plotly.express as px
import ipywidgets as widgets
pd.options.display.max_columns = 300
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from PIL import Image
import plotly.graph_objects as go

champion_roles = pd.read_csv("./data/champion_roles.csv", sep=",")

In [2]:
# https://www.kaggle.com/datasets/barthetur/league-of-legends-2024-competitive-game-dataset?resource=download
# https://oracleselixir.com/
df = pd.read_csv("./data/2024_LoL_esports_match_data_from_OraclesElixir_gamedata.csv", sep=";")
df = df[df["datacompleteness"] == "complete"]

# Drop all sets where bans are missing
bans = []
for nr in [1, 2]:
    for bannr in [1, 2, 3, 4, 5]:
        bans += [f"t{nr}_ban{bannr}"]

df = df.dropna(subset=bans)
df.sample(3)

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,gamelength,ckpm,t1_name,t1_id,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t1_result,t1_kills,t1_deaths,t1_kpm,t1_side,t1_doublekills,t1_triplekills,t1_quadrakills,t1_pentakills,t1_firstblood,t1_firstdragon,t1_dragons,t1_elementaldrakes,t1_infernals,t1_mountains,t1_clouds,t1_oceans,t1_chemtechs,t1_hextechs,t1_elders,t1_firstherald,t1_heralds,t1_firstbaron,t1_barons,t1_firsttower,t1_towers,t1_firstmidtower,t1_firsttothreetowers,t1_turretplates,t1_inhibitors,t1_damagetochampions,t1_dpm,t1_damageshare,t1_damagetakenperminute,t1_damagemitigatedperminute,t1_wardsplaced,t1_wpm,t1_wardskilled,t1_wcpm,t1_controlwardsbought,t1_visionscore,t1_vspm,t1_totalgold,t1_earnedgold,t1_earned_gpm,t1_earnedgoldshare,t1_goldspent,t1_gspd,t1_totalcs,t1_minionkills,t1_monsterkills,t1_monsterkillsownjungle,t1_monsterkillsenemyjungle,t1_cspm,t1_goldat10,t1_xpat10,t1_csat10,t1_golddiffat10,t1_xpdiffat10,t1_csdiffat10,t1_killsat10,t1_assistsat10,t1_deathsat10,t1_goldat15,t1_xpat15,t1_csat15,t1_golddiffat15,t1_xpdiffat15,t1_csdiffat15,t1_killsat15,t1_assistsat15,t1_deathsat15,t2_name,t2_id,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5,t2_result,t2_kills,t2_deaths,t2_kpm,t2_side,t2_doublekills,t2_triplekills,t2_quadrakills,t2_pentakills,t2_firstblood,t2_firstdragon,t2_dragons,t2_elementaldrakes,t2_elementaldrakes.1,t2_infernals,t2_mountains,t2_clouds,t2_oceans,t2_chemtechs,t2_hextechs,t2_elders,t2_firstherald,t2_heralds,t2_firstbaron,t2_barons,t2_firsttower,t2_towers,t2_firstmidtower,t2_firsttothreetowers,t2_turretplates,t2_inhibitors,t2_damagetochampions,t2_dpm,t2_damagetakenperminute,t2_damagemitigatedperminute,t2_wardsplaced,t2_wpm,t2_wardskilled,t2_wcpm,t2_controlwardsbought,t2_visionscore,t2_vspm,t2_totalgold,t2_earnedgold,t2_earned_gpm,t2_earnedgoldshare,t2_goldspent,t2_gspd,t2_totalcs,t2_minionkills,t2_monsterkills,t2_monsterkillsownjungle,t2_monsterkillsenemyjungle,...,t2p3_earned_gpm,t2p3_earnedgoldshare,t2p3_goldspent,t2p3_gspd,t2p3_totalcs,t2p3_minionkills,t2p3_monsterkills,t2p3_monsterkillsownjungle,t2p3_monsterkillsenemyjungle,t2p3_cspm,t2p3_goldat10,t2p3_xpat10,t2p3_csat10,t2p3_golddiffat10,t2p3_xpdiffat10,t2p3_csdiffat10,t2p3_killsat10,t2p3_assistsat10,t2p3_deathsat10,t2p3_goldat15,t2p3_xpat15,t2p3_csat15,t2p3_golddiffat15,t2p3_xpdiffat15,t2p3_csdiffat15,t2p3_killsat15,t2p3_assistsat15,t2p3_deathsat15,t2p4_position,t2p4_playername,t2p4_playerid,t2p4_champion,t2p4_kills,t2p4_deaths,t2p4_assists,t2p4_doublekills,t2p4_triplekills,t2p4_quadrakills,t2p4_pentakills,t2p4_firstblood,t2p4_firstbloodkill,t2p4_firstbloodassist,t2p4_firstbloodvictim,t2p4_firstdragon,t2p4_barons,t2p4_firsttower,t2p4_inhibitors,t2p4_damagetochampions,t2p4_dpm,t2p4_damageshare,t2p4_damagetakenperminute,t2p4_damagemitigatedperminute,t2p4_wardsplaced,t2p4_wpm,t2p4_wardskilled,t2p4_wcpm,t2p4_controlwardsbought,t2p4_visionscore,t2p4_vspm,t2p4_totalgold,t2p4_earnedgold,t2p4_earned_gpm,t2p4_earnedgoldshare,t2p4_goldspent,t2p4_gspd,t2p4_totalcs,t2p4_minionkills,t2p4_monsterkills,t2p4_monsterkillsownjungle,t2p4_monsterkillsenemyjungle,t2p4_cspm,t2p4_goldat10,t2p4_xpat10,t2p4_csat10,t2p4_golddiffat10,t2p4_xpdiffat10,t2p4_csdiffat10,t2p4_killsat10,t2p4_assistsat10,t2p4_deathsat10,t2p4_goldat15,t2p4_xpat15,t2p4_csat15,t2p4_golddiffat15,t2p4_xpdiffat15,t2p4_csdiffat15,t2p4_killsat15,t2p4_assistsat15,t2p4_deathsat15,t2p5_position,t2p5_playername,t2p5_playerid,t2p5_champion,t2p5_kills,t2p5_deaths,t2p5_assists,t2p5_doublekills,t2p5_triplekills,t2p5_quadrakills,t2p5_pentakills,t2p5_firstblood,t2p5_firstbloodkill,t2p5_firstbloodassist,t2p5_firstbloodvictim,t2p5_firstdragon,t2p5_barons,t2p5_firsttower,t2p5_inhibitors,t2p5_damagetochampions,t2p5_dpm,t2p5_damageshare,t2p5_damagetakenperminute,t2p5_damagemitigatedperminute,t2p5_wardsplaced,t2p5_wpm,t2p5_wardskilled,t2p5_wcpm,t2p5_controlwardsbought,t2p5_visionscore,t2p5_vspm,t2p5_totalgold,t2p5_earnedgold,t2p5_earned_gpm,t2p5_earnedgoldshare,t2p5_goldspent,t2p5_gspd,t2p5_totalcs,t2p5_mi

In [3]:
team_columns = []
for column in ["name", "ban1", "ban2", "ban3", "ban4", "ban5"]:
    team_columns += ["t1_" + column, "t2_" + column]

for column in range(1, 6):
    column = str(column)
    team_columns += ["t1p" + column + "_champion", "t2p" + column + "_champion"]

df_reduced = df[["gameid", "datacompleteness", "league", "year", "split", "playoffs", "date", "game", "patch", "t1_result"] + team_columns]
df_reduced = df_reduced.dropna()
df_reduced = df_reduced.rename(columns=
                  {"t1p1_champion": "t1_top",
                   "t1p2_champion": "t1_jng",
                   "t1p3_champion": "t1_mid",
                   "t1p4_champion": "t1_adc",
                   "t1p5_champion": "t1_sup",
                   #
                   "t2p1_champion": "t2_top",
                   "t2p2_champion": "t2_top",
                   "t2p3_champion": "t2_top",
                   "t2p4_champion": "t2_top",
                   "t2p5_champion": "t2_top",
                   #
                   "t1_result": "result"
                   }
                  )
df_reduced.sample(3)

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,result,t1_name,t2_name,t1_ban1,t2_ban1,t1_ban2,t2_ban2,t1_ban3,t2_ban3,t1_ban4,t2_ban4,t1_ban5,t2_ban5,t1_top,t2_top,t1_jng,t2_top,t1_mid,t2_top,t1_adc,t2_top,t1_sup,t2_top
106,LOLTMNT03_31598,complete,LLA,2024,Opening,0,19/01/2024 00:15,1,14.01,1,Estral Esports,INFINITY,Nocturne,Ashe,LeBlanc,Lucian,Orianna,Xin Zhao,Heimerdinger,Vi,Poppy,Jarvan IV,Rumble,Udyr,Rell,Viego,Corki,Neeko,Kalista,Varus,Renata Glasc,Karma
918,LOLTMNT04_27083,complete,LPLOL,2024,Spring,0,08/02/2024 20:26,1,14.02,1,EGN Esports,Odivelas Sports Club,Ashe,Xin Zhao,Jayce,Kalista,Rumble,Karma,Yone,Jax,Tristana,LeBlanc,K'Sante,Poppy,Brand,Lillia,Twisted Fate,Ezreal,Senna,Varus,Nautilus,Maokai
972,LOLTMNT04_26271,complete,LEC,2024,Winter,1,10/02/2024 18:52,1,14.02,1,MAD Lions KOI,GiantX,Maokai,Varus,Xin Zhao,Brand,Kalista,Ivern,Blitzcrank,Zac,Rell,Taliyah,Gwen,K'Sante,Vi,Sejuani,Karma,Azir,Senna,Miss Fortune,Nautilus,Alistar


In [4]:

ban_columns = [f"t1_ban{i}" for i in range(1, 6)] + [f"t2_ban{i}" for i in range(1, 6)]
df_bans = df_reduced.melt(
    id_vars=["gameid", "league", "year", "split", "playoffs", "date", "game", "patch", "result", "t1_name", "t2_name"],
    value_vars=ban_columns,
    var_name="ban_type",
    value_name="ban"
)

# Add team column based on the ban_type
df_bans["team"] = df_bans["ban_type"].apply(lambda x: "t1" if "t1" in x else "t2")

# Drop unnecessary columns
df_bans = df_bans.drop(columns=["ban_type"])

# Final DataFrame
df_bans.sample(3)

,gameid,league,year,split,playoffs,date,game,patch,result,t1_name,t2_name,ban,team
3060,LOLTMNT01_48838,HM,2024,Spring,0,06/02/2024 15:26,1,14.02,1,GLORE,BRUTE,Graves,t1
841,LOLTMNT03_31152,LLA,2024,Opening,0,16/01/2024 22:53,2,14.01,0,Movistar R7,Estral Esports,Renata Glasc,t1
1332,LOLTMNT02_42116,HM,2024,Spring,0,02/02/2024 16:14,1,14.01,0,Europe Saviors Club,Team UNiTY,Neeko,t1


In [5]:
ban_counts = df_bans.groupby(["league", "team", "ban"]).size().reset_index(name="count")

In [6]:
top_bans = (
    ban_counts.sort_values(["league", "team", "count"], ascending=[True, True, False])
    .groupby(["league", "team"])
    .head(5)
)

# Create a Dash application
app = dash.Dash(__name__)

# Layout for the app
app.layout = html.Div([
    html.H1("Top 3 Bans per League"),
    
    # Dropdown to select a league
    dcc.Dropdown(
        id="league-dropdown",
        options=[{"label": league, "value": league} for league in df["league"].unique()],
        value=df["league"].unique()[0],  # Default value (first league)
        style={"width": "50%"}
    ),
    
    # Graphs to display the top bans for t1 and t2
    dcc.Graph(id="top-bans-graph-t1"),  # Correct id for t1 bans
    dcc.Graph(id="top-bans-graph-t2"),  # Correct id for t2 bans
])

@app.callback(
    [Output("top-bans-graph-t1", "figure"),
     Output("top-bans-graph-t2", "figure")],
    [Input("league-dropdown", "value")]
)

def update_graph(selected_league):
    # Filter the data based on the selected league
    filtered_df = top_bans[top_bans["league"] == selected_league]
    
    # Split into t1 and t2 dataframes
    filtered_df_t1 = filtered_df[filtered_df['team'] == 't1']
    filtered_df_t2 = filtered_df[filtered_df['team'] == 't2']
    
    def get_graph(filtered_df, selected_league, color):
        scale = filtered_df["count"].max()
        labels = {"ban": "Champion Ban", "count": "Ban Count", "team": "Team"}
        # Create figures for t1 and t2
        fig = px.bar(
            filtered_df,
            x="ban",
            y="count",
            color_discrete_sequence=[color],
            barmode="group",
            title=f"Top 3 Bans for {color.capitalize()} Side in {selected_league}",
            labels=labels
        )

        for x in fig.data[0].x:
            asset_name = "".join([c for c in x if c not in "' "])
            fig.add_layout_image(
                source=f"assets/champion/{asset_name}.png",
                x=x,
                y=scale/4,
                xref="x",
                yref="y",
                xanchor="center",
                sizex=scale/4,
                sizey=scale/4,
            )
        fig.update_layout(autosize=True)
        fig.update_layout(xaxis={"visible":False})
        return fig


    fig_t1 = get_graph(filtered_df_t1, selected_league, "royalblue")
    fig_t2 = get_graph(filtered_df_t2, selected_league, "indianred")
    return fig_t1, fig_t2

if __name__ == "__main__":
    app.run_server(debug=True)

In [7]:
ban_roles_df = ban_counts.merge(champion_roles, left_on="ban", right_on="Name")
# Roles we are interested in
roles = ['Top', 'Jungle', 'Middle', 'Bottom', 'Support']

# Calculate the ban count for each role within each league
league_ban_counts = ban_roles_df.groupby('league')[roles].sum()

# Calculate the total bans for each league
total_bans_per_league = league_ban_counts.sum(axis=1)

# Convert ban counts to percentages
league_ban_percentage = league_ban_counts.div(total_bans_per_league, axis=0) * 100

# Flatten the DataFrame for easy plotting
df_melted = league_ban_percentage.reset_index().melt(id_vars='league', value_vars=roles,
                                                     var_name='Role', value_name='Ban Percentage')

# Plot the data using Plotly
fig = px.bar(df_melted, x='league', y='Ban Percentage', color='Role',
             barmode='stack', title="Ban Percentages for Each Role by League",
             labels={'Ban Percentage': 'Ban Percentage (%)', 'league': 'League'})

# Show the plot
fig.show()

In [8]:
# Roles we are interested in
roles = ['Top', 'Jungle', 'Middle', 'Bottom', 'Support']

# Filter the data to include only the leagues you want (e.g., ['LEC', 'LPL'])
selected_leagues = ['LEC', 'LPL', 'LCK', 'LCS']  # Example list of leagues
df_filtered = ban_roles_df[ban_roles_df['league'].isin(selected_leagues)]

# Calculate the ban count for each role across all selected leagues
total_ban_counts = df_filtered[roles].sum()

# Calculate the total bans across all selected leagues (sum of all roles)
total_bans_all_roles = total_ban_counts.sum()

# Calculate the ban percentages for each role
ban_percentages = (total_ban_counts / total_bans_all_roles) * 100

# Create a DataFrame with the ban percentages
ban_percentage_df = pd.DataFrame(ban_percentages).reset_index()
ban_percentage_df.columns = ['Role', 'Ban Percentage']
ban_percentage_df = ban_percentage_df.round(2)
ban_percentage_df

# Create the figure
fig = go.Figure()

# Add the background image to the plot
fig.add_layout_image(
    dict(
        source="assets/map/map.png",
        x=0,
        y=1,
        xref="paper", yref="paper",
        sizex=1, sizey=1,
        opacity=1,
        layer="below"
    )
)

role_positions = {
    'Top': (0.02, 0.91),
    'Jungle': (0.04, 0.72),
    'Middle': (0.075, 0.55),
    'Bottom': (0.14, 0.235),
    'Support': (0.11, 0.37)
}

# Loop through the roles to add role images and text
for index, row in ban_percentage_df.iterrows():
    role = row['Role']
    ban_percentage = row['Ban Percentage']
    
    # Load the role image (ensure they are small enough or resize them)
    role_img = Image.open(f"assets/roles/{role}.png")
    role_img = role_img.resize((100, 100))  # Adjust the size as needed
    
    # Convert the role image to a base64 string
    img_data = role_img.convert("RGBA").tobytes("raw", "RGBA")

    # Get the custom position for the role image
    x, y = role_positions[role]
    
    # Add the role image to the figure
    fig.add_layout_image(
        dict(
            source=f"assets/roles/{role}.png",  # Add your role image URL or local file path
            x=x, y=y,
            xref="paper", yref="paper",
            sizex=0.1, sizey=0.1,  # Adjust size here
            opacity=1,
            layer="above"
        )
    )

    # Add the text with the ban percentage below the role image
    fig.add_annotation(
        x=x+0.011, y=y-0.135,  # Adjust position for the text
        text=f"{ban_percentage}%",
        showarrow=False,
        font=dict(size=12, color="white", family="Arial"),
        align="center",
        bgcolor="black",
        opacity=0.7,
        borderpad=4
    )

# Set up the layout for better visual appeal
fig.update_layout(
    title="<b>Ban Percentages for Each Role [Major Regions]</b>",
    xaxis=dict(range=[0, 1], showgrid=False, zeroline=False),
    yaxis=dict(range=[0, 1], showgrid=False, zeroline=False),
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
    showlegend=False
)

fig.update_layout(
    title_font_color="black",
)
# Show the plot#
fig.update_layout(xaxis={"visible":False}, yaxis={"visible":False})
fig.update_layout({"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"})
fig.show()

In [9]:
# 3 Biggest Target Bans

def rename_opponent(row):
    if row['team'] == "t1":
        row['opponent'] = row['t2_name']
    elif row['team'] == "t2":
        row['opponent'] = row['t1_name']
    return row

# Apply the function row by row
df_bans_team = df_bans[df_bans['league'].isin(['LEC', 'LPL', 'LCK', 'LCS'])].apply(rename_opponent, axis=1)
df_bans_team = df_bans_team.drop(columns=["t1_name", "t2_name"])

df_bans_team = df_bans_team.groupby(['opponent', 'ban']).size().reset_index(name='count')
df_bans_team = df_bans_team.sort_values(by='count', ascending=False)
df_bans_team = df_bans_team.rename(columns={"opponent": "team"})
df_bans_team = df_bans_team.reset_index(drop=True)
df_bans_team = df_bans_team.head(5)

fig = go.Figure()

# Add bars to the chart
fig.add_trace(go.Bar(
    x=df_bans_team['ban'],
    y=df_bans_team['count'],
    marker=dict(color='royalblue'),
    name='Ban Count'
))

for index, x in enumerate(fig.data[0].x):
        ban = "".join([c for c in x if c not in "' "])
        fig.add_layout_image(
            source=f"assets/champion/{ban}.png",
            x=x,
            y=4,
            xref="x",
            yref="y",
            xanchor="center",
            sizex=4,
            sizey=4,
        )

        fig.add_annotation(
            x=x,
            y=5,
            xref="x",
            yref="y",
            text=f"<b>{df_bans_team.iloc[index]['team']}<br><i>{x} </i></b>",
            align="center",
            showarrow=False,
            font=dict(color="black"),
            borderwidth=2,
            borderpad=4,
            opacity=0.8
        )


fig.update_layout(title=dict(text="Most Common Target Bans [Major Regions]"))
fig.update_layout(xaxis={"visible":False})
fig.show()

In [10]:
# Create a list of all unique champions banned in t1_ban1 to t2_ban5
target_ban_df = df_reduced.drop(columns=["t1_top","t2_top","t1_jng","t2_top","t1_mid","t2_top","t1_adc","t2_top", "t1_sup", "t2_top"])
target_ban_df = target_ban_df[target_ban_df['league'].isin(['LEC', 'LPL', 'LCK', 'LCS'])]
banned_champions = pd.unique(target_ban_df[['t1_ban1', 't2_ban1', 't1_ban2', 't2_ban2', 't1_ban3', 't2_ban3', 't1_ban4', 't2_ban4', 't1_ban5', 't2_ban5']].values.ravel())

# Display the final dataframe
rows_t2 = df_reduced[['gameid', 'datacompleteness', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'result', 't2_name', 't1_name', 't1_ban1', 't1_ban2', 't1_ban3', 't1_ban4', 't1_ban5']].copy()
rows_t1 = df_reduced[['gameid', 'datacompleteness', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'result', 't1_name', 't2_name', 't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4', 't2_ban5']].copy()

rows_t2 = rows_t2.rename(columns={"t1_ban1": "ban1", "t1_ban2": "ban2", "t1_ban3": "ban3", "t1_ban4": "ban4", "t1_ban5": "ban5", "t2_name": "team"}).drop(columns=["t1_name"])
rows_t2["side"] = "Red"

rows_t1 = rows_t1.rename(columns={"t2_ban1": "ban1", "t2_ban2": "ban2", "t2_ban3": "ban3", "t2_ban4": "ban4", "t2_ban5": "ban5", "t1_name": "team"}).drop(columns=["t2_name"])
rows_t1["side"] = "Blue"


# Concatenate the rows for t2 and t1
df_transformed = pd.concat([rows_t2, rows_t1], ignore_index=True)
df_transformed

banned_champions = set()
for index in range(1,6):
    banned_champions.update(tuple(df_transformed[f"ban{index}"].unique()))
banned_champions = list(banned_champions)
banned_champions.sort()

new_data = pd.DataFrame(0, index=df.index, columns=banned_champions)

# Concatenate the new columns to the original DataFrame
df_transformed = pd.concat([df_transformed, new_data], axis=1)
df_transformed = df_transformed.fillna(0.0)
df_transformed = df_transformed.drop(columns=["gameid", "datacompleteness", "league", "year", "split", "playoffs", "date", "game", "patch", "result"])
# "ban1", "ban2", "ban3", "ban4", "ban5"])

def increment_bans(row):
    # For each ban column (ban1 to ban5)
    for i in range(1, 6):
        champ = row[f'ban{i}']
        if champ != '':  # Ensure there is a champion to increment
            row[champ] = row.get(champ, 0) + 1  # Increment the ban count
    return row

df_transformed = df_transformed.apply(increment_bans, axis=1)
df_transformed = df_transformed.drop(columns=["ban1", "ban2", "ban3", "ban4", "ban5"])
df_transformed.sample(3)

,team,side,Aatrox,Ahri,Akali,Akshan,Alistar,Anivia,Annie,Aphelios,Ashe,Aurelion Sol,Azir,Bard,Bel'Veth,Blitzcrank,Brand,Braum,Briar,Caitlyn,Cassiopeia,Cho'Gath,Corki,Darius,Draven,Elise,Ezreal,Fiora,Galio,Garen,Gnar,Gragas,Graves,Gwen,Hecarim,Heimerdinger,Hwei,Illaoi,Irelia,Ivern,Janna,Jarvan IV,Jax,Jayce,Jhin,Jinx,K'Sante,Kai'Sa,Kalista,Karma,Karthus,Kassadin,Kennen,Kha'Zix,Kindred,Kled,Kog'Maw,LeBlanc,Lee Sin,Leona,Lillia,Lissandra,Lucian,Lulu,Lux,Malphite,Maokai,Milio,Miss Fortune,Mordekaiser,Naafiri,Nautilus,Neeko,Nilah,Nocturne,Olaf,Orianna,Ornn,Poppy,Pyke,Rakan,Rell,Renata Glasc,Renekton,Rengar,Riven,Rumble,Ryze,Samira,Sejuani,Senna,Seraphine,Sett,Sion,Sivir,Skarner,Smolder,Sona,Soraka,Swain,Sylas,Syndra,Tahm Kench,Taliyah,Thresh,Tristana,Trundle,Tryndamere,Twisted Fate,Udyr,Urgot,Varus,Vayne,Vel'Koz,Vi,Viego,Viktor,Vladimir,Volibear,Wukong,Xayah,Xerath,Xin Zhao,Yasuo,Yone,Zac,Zed,Zeri,Ziggs,Zyra
1056,Team Secret,Blue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
904,Beyond Gaming,Blue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
419,WLGaming Esports,Red,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
ban_rate_per_team_df = df_transformed.drop(columns=["side"]).groupby(by=["team"]).mean().reset_index()

ban_rate_per_team_df = pd.melt(ban_rate_per_team_df, id_vars=['team'], var_name='champion', value_name='ban rate')
ban_rate_per_team_df = ban_rate_per_team_df.sort_values(by="ban rate", ascending=False)
ban_rate_per_team_df
major_teams = df_reduced[df_reduced['league'].isin(['LEC', 'LPL', 'LCK', 'LCS'])]["t1_name"].unique()  # Every team in t1 is also in t2
ban_rate_per_team_df = ban_rate_per_team_df[ban_rate_per_team_df["team"].isin(major_teams)]
ban_rate_per_team_df = ban_rate_per_team_df.head(5)

fig = px.bar(
    ban_rate_per_team_df,
    x="champion",
    y="ban rate",
    color_discrete_sequence=["royalblue"],
    title="Highest Ban-Rate Champions against Teams",
    labels={"champion": "Champion", "ban_rate": "Ban Rate", "team_name": "Team"},
    text="ban rate"
)

for index, x in enumerate(fig.data[0].x):
        ban = "".join([c for c in x if c not in "' "])
        fig.add_layout_image(
            source=f"assets/champion/{ban}.png",
            x=x, y=0.205,
            xref="x", yref="y",
            xanchor="center",
            sizex=0.25, sizey=0.25,
        )
        fig.add_annotation(
                    x=x,
                    y=0.28,
                    xref="x",
                    yref="y",
                    text=f"<b>{ban_rate_per_team_df.iloc[index]['team']}<br><i>{x} </i></b>",
                    align="center",
                    showarrow=False,
                    font=dict(color="black"),
                    borderwidth=2,
                    borderpad=4,
                    opacity=0.8
                )

fig.update_layout(xaxis={"visible":False})

fig.update_traces(texttemplate='%{text:.2%}', textposition='inside')
fig.update_layout(xaxis_title="Champion", yaxis_title="Ban Rate", legend_title="Team")
fig.show()